<a href="https://colab.research.google.com/github/bbenip/tetris-ai/blob/main/model/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/bbenip/tetris-ai.git

Mounted at /content/drive
Cloning into 'tetris-ai'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 137 (delta 17), reused 9 (delta 2), pack-reused 93
Receiving objects: 100% (137/137), 36.42 MiB | 26.51 MiB/s, done.
Resolving deltas: 100% (49/49), done.


Run data pre-proccessing, extracting json data for games.rar and building data.hdf5

In [115]:
!pip install tensorflow-io
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.6 MB/s eta 0:00:00


In [3]:
%run /content/tetris-ai/data/modify.ipynb

'Processing file: 8171/8172'

<HDF5 dataset "X_active_block": shape (2558757,), type "<i8">
<HDF5 dataset "X_block_pos": shape (2558757, 2), type "<i8">
<HDF5 dataset "X_block_rot": shape (2558757,), type "<i8">
<HDF5 dataset "X_grid": shape (2558757, 20, 10), type "|i1">
<HDF5 dataset "X_held_block": shape (2558757,), type "<i8">
<HDF5 dataset "Y": shape (2558757,), type "<i8">
<HDF5 dataset "X_active_block": shape (288380,), type "<i8">
<HDF5 dataset "X_block_pos": shape (288380, 2), type "<i8">
<HDF5 dataset "X_block_rot": shape (288380,), type "<i8">
<HDF5 dataset "X_grid": shape (288380, 20, 10), type "|i1">
<HDF5 dataset "X_held_block": shape (288380,), type "<i8">
<HDF5 dataset "Y": shape (288380,), type "<i8">
<HDF5 dataset "X_active_block": shape (2558757,), type "<i8">
<HDF5 dataset "X_block_pos": shape (2558757, 2), type "<i8">
<HDF5 dataset "X_block_rot": shape (2558757,), type "<i8">
<HDF5 dataset "X_grid": shape (2558757, 20, 10), type "|i1">
<HDF5 dataset "X_held_block": shape (2558757,), type "<i8">

Read in dataset

In [113]:
import tensorflow as tf
import tensorflow_io as tfio
import os

DATA_DIR = "/content/tetris-ai/data"
TRAIN_DATASET_FILE_NAME = "train_data.hdf5"
TEST_DATASET_FILE_NAME = "test_data.hdf5"

train_data_file = os.path.join(DATA_DIR, TRAIN_DATASET_FILE_NAME)
test_data_file = os.path.join(DATA_DIR, TEST_DATASET_FILE_NAME)
dataset_names = ['/X_held_block', '/X_active_block', '/X_block_pos', '/X_block_rot','/X_grid','/Y']

train_data = [tfio.IODataset.from_hdf5(train_data_file, setName) for setName in dataset_names]
test_data = [tfio.IODataset.from_hdf5(test_data_file, setName)  for setName in dataset_names]

In [114]:
print(test_data)

[<HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(2,), dtype=tf.int64, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(20, 10), dtype=tf.int8, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>]


In [115]:
#import data/data.hdf5
import h5py
import numpy as np
import os
from sklearn.model_selection import train_test_split

train_held, train_active, train_pos, train_rot, train_grid, train_y = train_data
test_held, test_active, test_pos, test_rot, test_grid, test_y = test_data

In [116]:
print(train_held)

<HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


Convert scalar datapoints (heldBlock, activeBlock, blockRotation, nextMove) into 1-hot vector representations

e.g.

>

>Currently Held Block ⟶ **4**<br>
>Number of Block Types ⟶ **8** (including 0 for None)<br>
>1-hot vector ⟶ **[0, 0, 0, 0, 1, 0, 0, 0]**<br>



In [117]:
train_held = train_held.map(lambda x: tf.one_hot(x, 8), num_parallel_calls=tf.data.AUTOTUNE)
train_active = train_active.map(lambda x: tf.one_hot(x, 8), num_parallel_calls=tf.data.AUTOTUNE)
train_rot = train_rot.map(lambda x: tf.one_hot(x, 4), num_parallel_calls=tf.data.AUTOTUNE)
train_y = train_y.map(lambda x: tf.one_hot(x, 10), num_parallel_calls=tf.data.AUTOTUNE)

print(train_held)

<_ParallelMapDataset element_spec=TensorSpec(shape=(8,), dtype=tf.float32, name=None)>


In [118]:
test_held = test_held.map(lambda x: tf.one_hot(x, 8), num_parallel_calls=tf.data.AUTOTUNE)
test_active = test_active.map(lambda x: tf.one_hot(x, 8), num_parallel_calls=tf.data.AUTOTUNE)
test_rot = test_rot.map(lambda x: tf.one_hot(x, 4), num_parallel_calls=tf.data.AUTOTUNE)
test_y =test_y.map(lambda x: tf.one_hot(x, 10), num_parallel_calls=tf.data.AUTOTUNE)

print(test_held)

<_ParallelMapDataset element_spec=TensorSpec(shape=(8,), dtype=tf.float32, name=None)>


In [119]:
def replace_non_zero_with_one(element):
    condition = tf.math.not_equal(element, 0)
    return tf.where(condition, np.int8(1), element)

In [120]:
train_grid = train_grid.map(replace_non_zero_with_one, num_parallel_calls=tf.data.AUTOTUNE)
test_grid = test_grid.map(replace_non_zero_with_one, num_parallel_calls=tf.data.AUTOTUNE)

Combine Datasets into one

In [121]:
trainDataset = tf.data.Dataset.zip((train_held, train_active, train_pos, train_rot, train_grid), train_y)
testDataset = tf.data.Dataset.zip((test_held, test_active, test_pos, test_rot, test_grid), test_y)

In [122]:
testDataset = testDataset.batch(128)

Train Model

In [123]:
# @title Training Constants { run: "auto" }
# @markdown Number of iterations over whole dataset
EPOCHS = 10 # @param {type:"integer"}

In [124]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, concatenate
from tensorflow.keras.models import Model

#Pass in hp to tune hyperparameters (# of nodes, activation function)
def build_model(hp):
  # Define input layers for each type of input
  cnn_input = Input((20,10,1))
  heldBlock_input = Input((8))
  activeBlock_input = Input((8))
  rotation_input = Input((4))
  position_input = Input((2))

  x = Conv2D(hp.Int('cnn_units_1', min_value=32, max_value=512, step=32), (5, 5), strides=(5,5), padding="same", activation=hp.Choice('activation', values=['relu', 'sigmoid', 'tanh']))(cnn_input)
  x = Conv2D(hp.Int('cnn_units_2', min_value=32, max_value=512, step=32), (3, 3), strides=(2,2), padding="same", activation=hp.Choice('activation', values=['relu', 'sigmoid', 'tanh']))(x)
  x = Conv2D(hp.Int('cnn_units_3', min_value=32, max_value=512, step=32), (3, 3), padding="same", activation=hp.Choice('activation', values=['relu', 'sigmoid', 'tanh']))(x)
  x = Flatten()(x)

  # Concatenate all the branches
  combined = concatenate([x, heldBlock_input, activeBlock_input, rotation_input, position_input])
  # Add more layers if needed
  combined = Dense(hp.Int('dense_units_4', min_value=32, max_value=512, step=32), activation=hp.Choice('activation', values=['relu', 'sigmoid', 'tanh']))(combined)
  combined = Dense(hp.Int('dense_units_5', min_value=32, max_value=512, step=32), activation=hp.Choice('activation', values=['relu', 'sigmoid', 'tanh']))(combined)
  combined = Dense(hp.Int('dense_units_6', min_value=32, max_value=512, step=32), activation=hp.Choice('activation', values=['relu', 'sigmoid', 'tanh']))(combined)
  output = Dense(10, activation='softmax')(combined)  # Replace 'output_dim' with the number of output classes or neurons

  # Create the model
  model = Model(inputs=[heldBlock_input, activeBlock_input, position_input, rotation_input, cnn_input], outputs=output)
  from keras.optimizers import Adam
  # Compile the model
  model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

  return model


Search for best hyperparameters

In [125]:
# @title  { form-width: "256px" }
# @title  { form-width: "128px" }
# @title  { run: "auto", form-width: "30px" }
from kerastuner.tuners import Hyperband

overwrite = True # @param {type:"boolean"}

tuner = Hyperband(
    hypermodel=build_model,
    objective='val_accuracy',
    overwrite=overwrite,
    max_epochs=EPOCHS,
    directory='/content/tuning',  # Directory to save the results
    project_name='tetris_tuning'
)

In [ ]:
tuner.search(
    x=trainDataset.shard(num_shards=8, index=0).batch(256).prefetch(tf.data.AUTOTUNE).cache(),\
    epochs=EPOCHS,\
    validation_data=testDataset,\
    use_multiprocessing=True
)

Trial 20 Complete [00h 03m 48s]
val_accuracy: 0.3106907606124878

Best val_accuracy So Far: 0.3174145221710205
Total elapsed time: 00h 55m 38s

Search: Running Trial #21

Value             |Best Value So Far |Hyperparameter
96                |512               |cnn_units_1
tanh              |tanh              |activation
448               |160               |cnn_units_2
160               |288               |cnn_units_3
64                |512               |dense_units_4
160               |64                |dense_units_5
160               |64                |dense_units_6
4                 |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/4
1250/1250 [==============================] - 44s 33ms/step - loss: 1.5406 - accuracy: 0.3008 - val_loss: 1.4979 - val_accuracy: 0.2999
Epoch 2/4
1249/1250 [============================>.] - ETA: 0s -

In [ ]:
#Build model using optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = build_model(best_hps)

In [ ]:
history = best_model.fit(
    trainDataset.batch(256).prefetch(tf.data.AUTOTUNE),\
    epochs=25,\
    validation_data=testDataset,\
    use_multiprocessing=True,\
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)]
)

In [ ]:
hf1.close()
hf2.close()

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()